In [ ]:
from google.colab import userdata

# Access the key by its name
mistral_api_key = userdata.get('Mistral_API')

# Use the api_key variable in your API call
if mistral_api_key is not None:
    # Your API call code here
    pass
else:
    print("API key not found in Colab secrets.")

In [ ]:
from langchain_core.tools import tool

In [ ]:
@tool
def find_sum(x:int, y:int)->int:
  """This function is used to sum two numbers and return their product
  It takes two integers as inputs and returns an integer as output"""
  return x+y


In [ ]:
@tool
def find_product(x:int, y:int)->int:
  """This function is used to multiply two numbers and return their products
  It takes two integers as inputs and return as integer as output"""
  return x*y

In [ ]:
!pip install -qU langchain-mistralai

In [ ]:
from langchain_mistralai import ChatMistralAI
models = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0,
    api_key=mistral_api_key


)

In [ ]:
from langchain_core.messages import AIMessage,HumanMessage, SystemMessage


In [ ]:
agent_tools = [find_sum,find_product]

In [ ]:
system_prompt = """You are a Math Genius who can solve math problems. Solve the problems provided by the user, by using only tools available, Do not solve the problem yourself"""

In [ ]:
from langchain.agents import create_agent

In [ ]:
agent = create_agent(
    model=models,
    system_prompt=system_prompt,
    tools = agent_tools
)

In [ ]:
input = {"messages":[("user","What is the sum of 2 and 3?")]}
result = agent.invoke(input)

In [ ]:
print(result['messages'][-1].content)

The sum of 2 and 3 is **5**.


In [ ]:
print("Step by Step Execution")
for message in result['messages']:
  print(message.pretty_repr())


Step by Step Execution
================================ Human Message =================================

What is the sum of 2 and 3?
================================== Ai Message ==================================
Tool Calls:
  find_sum (r0leCynuo)
 Call ID: r0leCynuo
  Args:
    x: 2
    y: 3
================================= Tool Message =================================
Name: find_sum

5
================================== Ai Message ==================================

The sum of 2 and 3 is **5**.


In [ ]:
input = {"messages":[("user","What is the 3 multiplied by 2 and 5+1")]}
result = agent.invoke(input)

In [ ]:
print(result['messages'][-1].content)
print("Step by Step Execution")
for message in result['messages']:
  print(message.pretty_repr())


The product of 3 multiplied by 2 is **6**.

The sum of 5 and 1 is **6**.
Step by Step Execution
================================ Human Message =================================

What is the 3 multiplied by 2 and 5+1
================================== Ai Message ==================================
Tool Calls:
  find_product (PK4DoZsMb)
 Call ID: PK4DoZsMb
  Args:
    x: 3
    y: 2
  find_sum (dCA1HOvBJ)
 Call ID: dCA1HOvBJ
  Args:
    x: 5
    y: 1
================================= Tool Message =================================
Name: find_product

6
================================= Tool Message =================================
Name: find_sum

6
================================== Ai Message ==================================

The product of 3 multiplied by 2 is **6**.

The sum of 5 and 1 is **6**.


In [ ]:
from langchain_mistralai import MistralAIEmbeddings
embeddings = MistralAIEmbeddings(
    model = "mistral-embed",
    api_key=mistral_api_key
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# Add product pricing function tool

In [ ]:
import pandas as pd
from langchain_core.tools import tool

In [ ]:
product_pricing_df = pd.read_csv("/content/drive/MyDrive/Data_Sets/RAG_and_AgenticRAG/Laptop pricing.csv")

In [ ]:
print(product_pricing_df)

            Name  Price  ShippingDays
0  AlphaBook Pro   1499             2
1     GammaAir X   1399             7
2  SpectraBook S   2499             7
3   OmegaPro G17   2199            14
4  NanoEdge Flex   1699             2


In [ ]:
@tool
def get_laptop_price(laptop_name : str)->int:
  """This function returns the price of the laptop, if you send it's name as input"""
  matched_record_df = product_pricing_df[product_pricing_df['Name'].str.contains("^"+laptop_name, case = False)]
  if len(matched_record_df) == 0:
    return -1
  else:
    return matched_record_df['Price'].iloc[0]


In [ ]:
!pip install -qU "langchain-chroma>=0.1.2"
!pip install langchain_community
!pip install pypdf

  Using cached pypdf-6.5.0-py3-none-any.whl.metadata (7.1 kB)
Using cached pypdf-6.5.0-py3-none-any.whl (329 kB)


In [ ]:
from langchain.tools import tool
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
loader = PyPDFLoader("/content/drive/MyDrive/Data_Sets/RAG_and_AgenticRAG/Laptop product descriptions.pdf")

In [ ]:
docs = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
splits = text_splitter.split_documents(docs)

In [ ]:
#Create a vector store with Chroma
prod_feature_store = Chroma.from_documents(
    documents=splits,
    embedding=embeddings
)

In [ ]:
retriever = prod_feature_store.as_retriever()

In [ ]:
@tool
def get_product_features(query: str) -> str:
  """Search and return information about given product"""
  docs = retriever.invoke(query)
  return "\n\n".join([doc.page_content for doc in docs])

In [ ]:
from langchain.agents import create_agent

In [ ]:
from langgraph.store.memory import InMemoryStore
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage


In [ ]:
system_prompt = SystemMessage("""
    You are professional chatbot that answers questions about laptops sold by your company.
    To answer questions about laptops, you will ONLY use the available tools and NOT your own memory.
    You will handle small talk and greetings by producing professional responses.
    """
)

In [ ]:
tools = [get_laptop_price, get_product_features]

In [ ]:
checkpointer=MemorySaver()


In [ ]:
#Create a Product QnA Agent. This is actual a graph in langGraph
product_QnA_agent=create_agent(
                                model=models, #LLM to use
                                tools=tools, #List of tools to use
                                system_prompt=system_prompt, #The system prompt
                                debug=False, #Debugging turned on if needed
                                checkpointer=checkpointer #For conversation memory
)

In [ ]:
import uuid
#To maintain memory, each request should be in the context of a thread.
#Each user conversation will use a separate thread ID
config = {"configurable": {"thread_id": uuid.uuid4()}}

#Test the agent with an input
inputs = {"messages":[
                HumanMessage("What are the features and pricing for GammaAir?")
            ]}

In [ ]:

#This is an alternate way to stream agent responses without waiting for the agent to finish
for stream in product_QnA_agent.stream(inputs, config, stream_mode="values"):
    message=stream["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

What are the features and pricing for GammaAir?
================================== Ai Message ==================================
Tool Calls:
  get_laptop_price (rSmPJMBZW)
 Call ID: rSmPJMBZW
  Args:
    laptop_name: GammaAir
  get_product_features (6167tv16K)
 Call ID: 6167tv16K
  Args:
    query: GammaAir
================================= Tool Message =================================
Name: get_product_features

OmegaPro G17
OmegaPro G17 is a gaming powerhouse with a Ryzen 9 5900HX CPU, 32GB RAM, and a 1TB
SSD. Designed for gamers, it features a 17-inch display with a high refresh rate and powerful
graphics card for the best gaming experience.
NanoEdge Flex
NanoEdge Flex is a versatile 2-in-1 laptop with Apple's M1 Pro chip, 16GB of unified memory, and a
512GB SSD. It's perfect for creative professionals who need a device that can transform between a
laptop and a tablet.

Fictional Laptop Descriptions
A

In [ ]:
import uuid
#Send a sequence of messages to chatbot and get its response
#This simulates the conversation between the user and the Agentic chatbot
user_inputs = [
    "Hello",
    "I am looking to buy a laptop",
    "Give me a list of available laptop names",
    "Tell me about the features of  SpectraBook",
    "How much does it cost?",
    "Give me similar information about OmegaPro",
    "What info do you have on AcmeRight ?",
    "Thanks for the help"
]


In [ ]:
#Create a new thread
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

for input in user_inputs:
    print(f"----------------------------------------\nUSER : {input}")
    #Format the user message
    user_message = {"messages":[HumanMessage(input)]}
    #Get response from the agent
    ai_response = product_QnA_agent.invoke(user_message,config=config)
    #Print the response
    print(f"AGENT : {ai_response['messages'][-1].content}")


----------------------------------------
USER : Hello
AGENT : Hello! How can I assist you today? Are you looking for information about any of our laptops?
----------------------------------------
USER : I am looking to buy a laptop
AGENT : Great! We have a variety of laptops to suit different needs—whether it's for work, gaming, study, or everyday use.

Could you let me know what you're looking for in a laptop? For example:
- **Budget range**
- **Preferred brand or specifications** (e.g., RAM, storage, processor)
- **Primary use** (e.g., gaming, programming, design, general use)
- **Any specific features** (e.g., lightweight, touchscreen, long battery life)
----------------------------------------
USER : Give me a list of available laptop names
AGENT : I currently don't have a direct list of all available laptop names, but I can help you find information about specific laptops if you let me know what you're interested in.

For example, if you're looking for laptops like:
- **Dell XPS 1

In [ ]:
#conversation memory by user
def execute_prompt(user, config, prompt):
    inputs = {"messages":[("user",prompt)]}
    ai_response = product_QnA_agent.invoke(inputs,config=config)
    print(f"\n{user}: {ai_response['messages'][-1].content}")

#Create different session threads for 2 users
config_1 = {"configurable": {"thread_id": str(uuid.uuid4())}}
config_2 = {"configurable": {"thread_id": str(uuid.uuid4())}}

#Test both threads
execute_prompt("USER 1", config_1, "Tell me about the features of  SpectraBook")
execute_prompt("USER 2", config_2, "Tell me about the features of  GammaAir")
execute_prompt("USER 1", config_1, "What is its price ?")
execute_prompt("USER 2", config_2, "What is its price ?")




USER 1: Here are the features of the **SpectraBook S**:

- **Processor**: Intel Core i9 (workstation-class performance)
- **RAM**: 64GB (ideal for intensive multitasking)
- **Storage**: 2TB SSD (massive storage for large files and applications)
- **Use Case**: Perfect for power users, video editing, 3D rendering, and other demanding tasks.

USER 2: Here are the features of the **GammaAir X** laptop:

- **Processor:** AMD Ryzen 7 (high-performance processing)
- **RAM:** 32GB DDR4 (supports multitasking and smooth performance)
- **Storage:** 512GB NVMe SSD (fast storage for quick boot and load times)
- **Design:** Thin and light form factor (highly portable)
- **Best For:** Users who need **high performance** in a **portable** design.

Would you like to know its price or compare it with another laptop?

USER 1: The **SpectraBook S** is priced at **$2,499**.

USER 2: The **GammaAir X** is priced at **$1,399**.

Would you like assistance with anything else, such as comparisons or availabi